### Prosecution severity
##### Data:
1. case_offense_v01.csv (merged in bonds_analysis_01.ipynb, but ideally can be merged anywhere?)
2. ACIS_offence_codes_FIXED.csv (from the 00_raw_data folder that Clarissa emailed)
 
##### Steps
1. Load Data 
    * Load the files listed above
 
2. Merge charged and convicted codes with code descriptions
    * Rename columns in the ACIS_offence_codes_FIXED df (offense_desc_final) with the charged prefix and merge with offenses. 
    * Rename columns in the ACIS_offence_codes_FIXED df (offense_desc_final) with the conv prefix and merge with previous output. 
 
3. Data Quality Fixes
    * Fix Offense class strings ("1.0" to "1" etc.)
    * Filter for gender values (M/F/U). Drop X ~500 cases. 
 
4.  Determining Delta
    * Based on offense class
        * "delta_min" is the difference between "conv_CL_Rank_min" and "CL_Rank", i.e. The minimum convicted offense
        * "delta_max" is the difference between "conv_CL_Rank_max" and "CL_rank", i.e. The maximum convicted offense
    * In cases where the charged offense code matches the convicted offense code, set delta_min and delta_max to zero

#### 1. Load Data (takes ~4 mins)

In [2]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None) 
# I am blind without this
%config InlineBackend.figure_format = 'retina' 

### Load the required files
file_dir = "/workspaces/esg-controversy-tracker/nc_acis-main/20_intermediate_data"
offenses = pd.read_csv(os.path.join(file_dir, "case_offense_v01.csv"), low_memory=False, index_col=[0])
offense_desc_final = pd.read_csv(os.path.join(file_dir,'ACIS_offence_codes_FIXED.csv'))

# subset
offs = offenses[
    [
        "case_id",
        "date_of_birth",
        "key_county_num",
        "key_year",
        "court_type",
        "race",
        "sex",
        "process_served",
        "case_creation_date",
        "case_trial_date",
        "court_attorney_type",
        "bond_type",
        "bond_amount",
        "charged_offense_date",
        "charged_offense_code",
        "offense_class",
        "min_sentence",
        "convicted_offense_code",
        "disposition",
        "disposition_date",
    ]
]

offs.head(5)

,case_id,date_of_birth,key_county_num,key_year,court_type,race,sex,process_served,case_creation_date,case_trial_date,court_attorney_type,bond_type,bond_amount,charged_offense_date,charged_offense_code,offense_class,min_sentence,convicted_offense_code,disposition,disposition_date
0,1973006239,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,10000.0,1973-05-20,3599,NaN,NaN,NaN,SI,1973-08-07
1,1973006239,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,10000.0,1973-05-20,3599,NaN,NaN,NaN,VD,2013-10-18
2,1973007828,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,1.0,1973-05-19,3599,NaN,NaN,NaN,SI,1973-08-07
3,1973007828,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,1.0,1973-05-19,3599,NaN,NaN,NaN,VD,2013-10-18
4,1973013840,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,3000.0,1973-10-13,3599,NaN,NaN,NaN,SI,1974-01-22


### 2. Merge Charged and Convicted codes with Code Descriptions

In [3]:
# rename cols
charged_offense_desc = offense_desc_final.rename(
    columns={
        "Offense Description": "charg_Off_Desc",
        "CODE": "charg_CODE",
        "T": "charg_type",
        "NC General Statute": "charg_statute",
        "CL_min": "charg_CL_min",
        "CL_Rank_min": "charg_CL_Rank_min",
        "CL_max": "charg_CL_max",
        "CL_Rank_max": "charg_CL_Rank_max",
    },
)

# merge
offs_rank = pd.merge(
    left=offs,
    right=charged_offense_desc,
    left_on="charged_offense_code",
    right_on="charg_CODE",
    how="left",
    validate="m:m"
)

# Merge with offence code CONVICTED descriptions:
# rename cols
conv_offense_desc = offense_desc_final.rename(
    columns={
        "Offense Description": "conv_Off_Desc",
        "CODE": "conv_CODE",
        "T": "convc_type",
        "NC General Statute": "conv_statute",
        "CL_min": "conv_CL_min",
        "CL_Rank_min": "conv_CL_Rank_min",
        "CL_max": "conv_CL_max",
        "CL_Rank_max": "conv_CL_Rank_max",
    },
)

# merge
offs_rank = pd.merge(
    left=offs_rank,
    right=conv_offense_desc,
    left_on="convicted_offense_code",
    right_on="conv_CODE",
    how="left",
    validate="m:m"
)
offs_rank.head()

,case_id,date_of_birth,key_county_num,key_year,court_type,race,sex,process_served,case_creation_date,case_trial_date,court_attorney_type,bond_type,bond_amount,charged_offense_date,charged_offense_code,offense_class,min_sentence,convicted_offense_code,disposition,disposition_date,Unnamed: 0_x,charg_CODE,charg_Off_Desc,charg_type,charg_statute,charg_CL_min,charg_CL_Rank_min,charg_CL_max,charg_CL_Rank_max,Unnamed: 0_y,conv_CODE,conv_Off_Desc,convc_type,conv_statute,conv_CL_min,conv_CL_Rank_min,conv_CL_max,conv_CL_Rank_max
0,1973006239,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,10000.0,1973-05-20,3599,NaN,NaN,NaN,SI,1973-08-07,786.0,3599.0,DANGEROUS DRUGS - FREE TEXT,,,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1973006239,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,10000.0,1973-05-20,3599,NaN,NaN,NaN,VD,2013-10-18,786.0,3599.0,DANGEROUS DRUGS - FREE TEXT,,,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1973007828,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,1.0,1973-05-19,3599,NaN,NaN,NaN,SI,1973-08-07,786.0,3599.0,DANGEROUS DRUGS - FREE TEXT,,,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973007828,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,1.0,1973-05-19,3599,NaN,NaN,NaN,VD,2013-10-18,786.0,3599.0,DANGEROUS DRUGS - FREE TEXT,,,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1973013840,7f69fac81ed85a6a41996f079cf9adcbd1ea89098e756a...,0,73,CRS,W,M,W,2013-10-22,1974-05-09,R,SEC,3000.0,1973-10-13,3599,NaN,NaN,NaN,SI,1974-01-22,786.0,3599.0,DANGEROUS DRUGS - FREE TEXT,,,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Data Quality Fixes

In [4]:
# lets fix the Offense Class column
offs_rank.loc[offs_rank["offense_class"] == "1.0", "offense_class"] = "1"
offs_rank.loc[offs_rank["offense_class"] == "2.0", "offense_class"] = "2"
offs_rank.loc[offs_rank["offense_class"] == "3.0", "offense_class"] = "3"
offs_rank.loc[offs_rank["offense_class"] == "i", "offense_class"] = "I"

offs_rank.loc[offs_rank['offense_class']==" ", 'offense_class']=np.nan 
offs_rank.loc[offs_rank['offense_class']=="??", 'offense_class']=np.nan


# Clean up sex:
offs_rank.sex.value_counts()
offs_rank = offs_rank[offs_rank["sex"].isin(["M", "F", "U"])] # get rid of the X category, only 500 offenses 
offs_rank.sample(5)

,case_id,date_of_birth,key_county_num,key_year,court_type,race,sex,process_served,case_creation_date,case_trial_date,court_attorney_type,bond_type,bond_amount,charged_offense_date,charged_offense_code,offense_class,min_sentence,convicted_offense_code,disposition,disposition_date,Unnamed: 0_x,charg_CODE,charg_Off_Desc,charg_type,charg_statute,charg_CL_min,charg_CL_Rank_min,charg_CL_max,charg_CL_Rank_max,Unnamed: 0_y,conv_CODE,conv_Off_Desc,convc_type,conv_statute,conv_CL_min,conv_CL_Rank_min,conv_CL_max,conv_CL_Rank_max
3233090,1802017050780,6d44796f75781651b8a6f58717d1523889046481289b08...,180,17,CRS,B,M,W,2017-06-05,2018-07-26,NaN,SEC,1.0,2017-02-01,3487,NaN,NaN,NaN,SI,2017-07-17,533.0,3487.0,M/S/D/P CS W/N 1000FT SCHOOL,F,90-95(E)(8),E,9.0,E,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12847824,5902013048970,630115fa83bf6dd1549476134d2fa8fb7ba493c45ea8f4...,590,13,CR,B,F,C,2013-12-20,2014-01-31,NaN,NaN,NaN,2013-12-17,4440,NaN,NaN,NaN,VD,2014-01-30,1090.0,4440.0,EXPIRED/NO INSPECTION,I,20-183.8(A)(1),0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5446511,2902015051254,b0d7ab0ee96d4d179e7814825c1b1ba006c9a42b768b4c...,290,15,CR,W,M,M,2015-08-19,2016-02-18,R,SEC,1000.0,2015-08-19,4533,NaN,NaN,NaN,VD,2016-02-18,1177.0,4533.0,TEXTING VIOL MOTOR VEH,I,20-137.4A(A),0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11534953,5002020716116,620ce62f4e606cc471e7c3dd9c1be77700a8ae55af2d06...,500,20,CR,B,M,C,2020-12-21,2021-03-25,NaN,NaN,NaN,2020-12-21,4721,NaN,NaN,NaN,VD,2021-03-25,1288.0,4721.0,CANCL/REVOK/SUSP CERTIF/TAG,T,20-111(2),2,2.0,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8750329,4002013739986,bf35a38a6cd8280214b8d263ef9b217be666ca4bc30bbe...,400,13,CR,W,M,C,2013-12-27,2015-03-18,R,NaN,NaN,2013-12-27,4716,3,60.0,4716.0,JU,2015-03-18,1473.0,4716.0,DWLR NOT IMPAIRED REV,T,20-28(A),3,1.0,3,1.0,1473.0,4716.0,DWLR NOT IMPAIRED REV,T,20-28(A),3,1.0,3,1.0


### 4. Determining delta (difference between convicted vs. charged crimes)

In [5]:
# create ranks for classes
class_ranks = {
    "0": 0,
    "3": 1,
    "2": 2,
    "1": 3,
    "A1": 4,
    "I": 5,
    "H": 6,
    "G": 7,
    "F": 8,
    "E": 9,
    "D": 10,
    "C": 11,
    "B2": 12,
    "B1": 13,
    "A": 14,
    "??": np.nan,
}
class_ranks = pd.DataFrame.from_dict(class_ranks, "index").reset_index()
class_ranks.rename(columns={0: "CL_Rank"}, inplace=True)

#merge
offs_rank = pd.merge(offs_rank, class_ranks, left_on="offense_class", right_on="index", how="left")  # now, 'CL_rank' is the rank of the charged offense class provided by the court

# NOW lets look at deltas at the offense level for a second
offs_rank["delta_min"] = (offs_rank["conv_CL_Rank_min"] - offs_rank["CL_Rank"])  
#
# DELTA = final - initial, Delta MIN is the one using the minimum ranking of the convicted offense (when it can belong in two or more classes
# This means delta_min will be probably a larger delta than delta max
#
offs_rank["delta_max"] = offs_rank["conv_CL_Rank_max"] - offs_rank["CL_Rank"]

# if codes are the same, fill NaN with 0
offs_rank.loc[
    offs_rank["charged_offense_code"] == offs_rank["convicted_offense_code"],
    "delta_min",
] = 0
offs_rank.loc[
    offs_rank["charged_offense_code"] == offs_rank["convicted_offense_code"],
    "delta_max",
] = 0
offs_rank.sample(5)


,case_id,date_of_birth,key_county_num,key_year,court_type,race,sex,process_served,case_creation_date,case_trial_date,court_attorney_type,bond_type,bond_amount,charged_offense_date,charged_offense_code,offense_class,min_sentence,convicted_offense_code,disposition,disposition_date,Unnamed: 0_x,charg_CODE,charg_Off_Desc,charg_type,charg_statute,charg_CL_min,charg_CL_Rank_min,charg_CL_max,charg_CL_Rank_max,Unnamed: 0_y,conv_CODE,conv_Off_Desc,convc_type,conv_statute,conv_CL_min,conv_CL_Rank_min,conv_CL_max,conv_CL_Rank_max,index,CL_Rank,delta_min,delta_max
3403118,2202004008505,33b8e30b58c9dd06c35ebe21fe51e9a350cfd2b6492275...,220,4,CR,B,M,C,2004-11-29,2005-01-07,NaN,NaN,NaN,2004-11-25,5450,NaN,NaN,NaN,VD,2018-05-18,1300.0,5450.0,SPEEDING,T,20-141(J1),2,2.0,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22605535,9702011053212,dedcb9f59d6df777e475688350e2f487a90cf193087971...,970,11,CRS,B,M,W,2011-07-22,2017-03-27,W,NaN,NaN,2011-06-18,5032,NaN,NaN,NaN,PO,2015-10-26,1230.0,5032.0,FELONY PROBATION VIOLATION,F,15A-1345,??,NaN,??,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18142666,7802018701429,02ecbc4612372df0f9ab08f780e610a206ccc1ada470e1...,780,18,CR,O,F,C,2018-02-28,2018-05-17,NaN,NaN,NaN,2018-02-28,4470,NaN,NaN,NaN,VD,2018-05-17,1120.0,4470.0,FAIL TO WEAR SEAT BELT-DRIVER,I,20-135.2A,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18197910,7802020707019,0c200aeb498bd2cba95b0e8db04a90c7b80af016b4a7fe...,780,20,CR,B,M,C,2020-12-21,2021-04-08,R,NaN,NaN,2020-12-20,5461,NaN,NaN,NaN,VD,2021-04-08,1302.0,5461.0,EXPIRED REGISTRATION CARD/TAG,T,20-111(2),2,2.0,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13854485,5902019020220,da1e335e1aa4ccd1858684eeb591df12a7f4359002703c...,590,19,CR,B,M,C,2019-09-03,2020-01-16,R,NaN,NaN,2019-08-29,5461,NaN,NaN,NaN,VD,2020-01-16,1302.0,5461.0,EXPIRED REGISTRATION CARD/TAG,T,20-111(2),2,2.0,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
offs_rank['delta_min'].value_counts()

delta_min
 0.0     3294178
 1.0       29796
-1.0        2665
-2.0         582
 2.0         297
-3.0         216
-4.0          45
 3.0          20
-5.0          16
-7.0           3
-6.0           3
 5.0           2
 4.0           2
-14.0          1
Name: count, dtype: int64

In [7]:
offs_rank['delta_max'].value_counts()

delta_max
 0.0     3294045
-1.0       22752
 1.0        9945
-2.0         616
 2.0         302
-3.0         179
-4.0          46
 3.0          24
-5.0          14
-7.0           3
-6.0           3
 5.0           2
 4.0           2
-14.0          1
Name: count, dtype: int64